# Monte Carlo estimate: O‑type stars in 20,360–20,380 ly shell (spiral‑arm model)

This notebook samples points inside the spherical shell centered on the Sun (20,360–20,380 ly), converts to galactocentric coordinates, evaluates an exponential surface density for O stars with an exponential vertical profile, and determines whether points lie within spiral arms. It supports loading a published spiral-arm locus (e.g., a Reid+2014-style table) from a CSV named `reid_arms.csv` in the same folder. If the CSV is not found it falls back to a parametric logarithmic-arm model. The expected number of O stars inside the shell and inside arms is computed.

In [1]:
# --- Parameters for O-star distribution and spiral arms ---
r1_ly = 20360.0
r2_ly = 20380.0
r1 = r1_ly * 0.306601  # convert to pc
r2 = r2_ly * 0.306601  # convert to pc

import math
import numpy as np
import os

# Sun's galactocentric radius
R0_pc = 8122.0  # pc (8.122 kpc)

# Disk model (O stars)
Rd = 2600.0  # scale length (pc)
hz = 300.0   # vertical scale height (pc) — typical for massive stars
Rmax = 15000.0  # effective disk radius (pc)

# Spiral arms
n_arms = 4
pitch_deg = 12.0
arm_half_width = 300.0  # radial half-width of arms (pc)

# population
N_total_nominal = 30000.0  # nominal total O stars in Milky Way

# Monte Carlo samples (increased for tighter statistics)
N_mc = 250000

# --- helper: compute sigma0 such that integral over disk area gives N_total ---
def compute_sigma0(N_total, Rd, Rmax):
    integral = 2 * math.pi * Rd**2 * (1 - math.exp(-Rmax/Rd)*(1 + Rmax/Rd))
    return N_total / integral

sigma0 = compute_sigma0(N_total_nominal, Rd, Rmax)

# --- load Reid-style arms if available ---
arm_params = []
use_reid_csv = os.path.exists('reid_arms.csv')
if use_reid_csv:
    try:
        import csv
        with open('reid_arms.csv', newline='') as f:
            reader = csv.reader(f)
            next(reader)  # skip header
            for row in reader:
                if len(row) < 4:
                    continue
                name = row[0]
                R_ref = float(row[1])
                phi_ref = math.radians(float(row[2]))
                pitch = math.radians(float(row[3]))
                arm_params.append(dict(name=name, R_ref=R_ref, phi_ref=phi_ref, pitch=pitch))
        print('Loaded arm parameters from reid_arms.csv')
    except Exception as e:
        print('Failed to read reid_arms.csv, falling back to parametric arms:', e)
        arm_params = []

# fallback parametric arms if no CSV
if not arm_params:
    pitch = math.radians(pitch_deg)
    r_ref = R0_pc
    phi0 = 0.0
    for k in range(n_arms):
        phi_ref_k = phi0 + k * (2.0*math.pi / n_arms)
        arm_params.append(dict(name=f'arm{k+1}', R_ref=r_ref, phi_ref=phi_ref_k, pitch=pitch))

# --- Monte Carlo sample points uniformly in shell volume ---
u = np.random.uniform(r1**3, r2**3, size=N_mc)
r = u ** (1.0/3.0)
cos_theta = np.random.uniform(-1.0, 1.0, size=N_mc)
theta = np.arccos(cos_theta)
phi = np.random.uniform(0.0, 2.0*math.pi, size=N_mc)

# positions in Sun-centered Cartesian coordinates (pc)
x_sun = r * np.sin(theta) * np.cos(phi)
y_sun = r * np.sin(theta) * np.sin(phi)
z_sun = r * cos_theta

# convert to Galactocentric coordinates: place Sun at (R0, 0, 0)
X_gc = R0_pc + x_sun
Y_gc = y_sun
Z_gc = z_sun

R_gc = np.sqrt(X_gc**2 + Y_gc**2)
phi_gc = np.arctan2(Y_gc, X_gc)  # -pi..pi

# surface density at R (pc^-2)
sigma = sigma0 * np.exp(-R_gc / Rd)
# volumetric density using exponential vertical distribution: rho = sigma/(2*hz) * exp(-|z|/hz)
rho = sigma / (2.0 * hz) * np.exp(-np.abs(Z_gc) / hz)

# --- higher-fidelity arm membership: compute minimal radial separation using a local phi grid ---
phi_window_half = math.radians(6.0)  # search +/- 6 degrees around each point
n_phi_samples = 121
phi_offsets = np.linspace(-phi_window_half, phi_window_half, n_phi_samples)

sep_min = np.full(N_mc, np.inf)
for arm in arm_params:
    R_ref = arm['R_ref']
    phi_ref = arm['phi_ref']
    pitch = arm['pitch']
    tan_p = math.tan(pitch)
    phi_grid = (phi_gc[:, None] + phi_offsets[None, :])
    phi_rel = (phi_grid - phi_ref + math.pi) % (2.0*math.pi) - math.pi
    R_arm_grid = R_ref * np.exp(phi_rel * tan_p)
    radial_sep_grid = np.abs(R_gc[:, None] - R_arm_grid)
    sep_min = np.minimum(sep_min, np.min(radial_sep_grid, axis=1))

in_arm = sep_min < arm_half_width

# compute shell volume (full spherical shell)
V_shell = 4.0/3.0 * math.pi * (r2**3 - r1**3)

# expected numbers
mean_rho = np.mean(rho)
N_expected_shell = mean_rho * V_shell
mean_rho_in_arms = np.mean(rho * in_arm)
N_expected_arms = mean_rho_in_arms * V_shell

# also compute fraction of sampled points falling into arm region (geometric fraction, not density-weighted)
frac_points_in_arm = np.mean(in_arm)

# print results
print(f"Parameters: r1={r1:.1f} pc, r2={r2:.1f} pc, N_mc={N_mc}")
print(f"Disk params: R0={R0_pc} pc, Rd={Rd} pc, hz={hz} pc, sigma0={sigma0:.3e} pc^-2")
print(f"Arms used: {len(arm_params)} (loaded from CSV: {use_reid_csv})")
print(f"Arm half-width = {arm_half_width} pc")
print('---')
print(f"Shell volume = {V_shell:.3e} pc^3")
print(f"Mean volumetric density in shell = {mean_rho:.3e} pc^-3")
print(f"Expected O stars in shell (all) = {N_expected_shell:.3f}")
print(f"Density-weighted expected in arms = {N_expected_arms:.3f}")
print(f"Geometric fraction of sampled points in arm region = {frac_points_in_arm:.4f}")

# sensitivity sweep for N_total
for Ntot in [20000, 30000, 50000]:
    sigma0_loc = compute_sigma0(Ntot, Rd, Rmax)
    sigma_loc = sigma0_loc * np.exp(-R_gc / Rd)
    rho_loc = sigma_loc / (2.0 * hz) * np.exp(-np.abs(Z_gc) / hz)
    Nloc = np.mean(rho_loc * in_arm) * V_shell
    print(f"N_total={Ntot:5d} -> Expected in arms ≈ {Nloc:.2f}")

# collect results
result = dict(
    V_shell=V_shell,
    N_expected_shell=float(N_expected_shell),
    N_expected_arms=float(N_expected_arms),
    frac_points_in_arm=float(frac_points_in_arm),
    params=dict(R0_pc=R0_pc, Rd=Rd, hz=hz, arm_half_width=arm_half_width, use_reid_csv=use_reid_csv)
)

result

Loaded arm parameters from reid_arms.csv
Parameters: r1=6242.4 pc, r2=6248.5 pc, N_mc=250000
Disk params: R0=8122.0 pc, Rd=2600.0 pc, hz=300.0 pc, sigma0=7.216e-04 pc^-2
Arms used: 4 (loaded from CSV: True)
Arm half-width = 300.0 pc
---
Shell volume = 3.006e+09 pc^3
Mean volumetric density in shell = 5.103e-09 pc^-3
Expected O stars in shell (all) = 15.339
Density-weighted expected in arms = 2.124
Geometric fraction of sampled points in arm region = 0.2016
N_total=20000 -> Expected in arms ≈ 1.42
N_total=30000 -> Expected in arms ≈ 2.12
N_total=50000 -> Expected in arms ≈ 3.54
Parameters: r1=6242.4 pc, r2=6248.5 pc, N_mc=250000
Disk params: R0=8122.0 pc, Rd=2600.0 pc, hz=300.0 pc, sigma0=7.216e-04 pc^-2
Arms used: 4 (loaded from CSV: True)
Arm half-width = 300.0 pc
---
Shell volume = 3.006e+09 pc^3
Mean volumetric density in shell = 5.103e-09 pc^-3
Expected O stars in shell (all) = 15.339
Density-weighted expected in arms = 2.124
Geometric fraction of sampled points in arm region = 0.2

{'V_shell': 3005679391.450835,
 'N_expected_shell': 15.338776485452833,
 'N_expected_arms': 2.123977075048088,
 'frac_points_in_arm': 0.20158,
 'params': {'R0_pc': 8122.0,
  'Rd': 2600.0,
  'hz': 300.0,
  'arm_half_width': 300.0,
  'use_reid_csv': True}}